Importing Libraries

In [ ]:
import pandas as pd 
import numpy as np

In [ ]:
# Try with ISO-8859-1 (a common Excel export encoding)
df = pd.read_csv("OLympic praticipants.csv", encoding="ISO-8859-1")
print(df.head(10))

In [ ]:
print(df.columns)

Inspectind data

In [ ]:
print(df.info())
print(df.describe(include="all"))
print(df.isnull().sum())

Handle Missing Values

In [ ]:
# Make column names lowercase
df.columns = df.columns.str.lower()
# Now you can safely use lowercase everywhere
df[['gold', 'silver', 'bronze']] = df[['gold', 'silver', 'bronze']].fillna(0)
df['year'] = df['year'].astype('Int64')

In [ ]:
# Standardize athlete names
df['name'] = df['name'].replace({
    'P Sindhu': 'Pv Sindhu'
})
df['gender'] = df['gender'].replace({'F': 'W'})

Clean TEXT Data

In [ ]:
df['name'] = df['name'].str.strip().str.title()
df['sport'] = df['sport'].str.strip().str.title()
df['city']  = df['city'].str.strip().str.title()
print(df.head())

In [ ]:
print("📌 Unique Years:")
years_list = sorted(df['year'].dropna().unique().tolist())
print(years_list, "\n")
print(len(years_list))
# Unique Cities
print("📌 Unique Cities:")
city_list = sorted(df['city'].dropna().unique().tolist())
print(city_list, "\n")
print(len(city_list))
gender_list = sorted(df['gender'].dropna().unique().tolist())
print("📌 Unique Genders:")
print(gender_list)
print("Total Genders:", len(gender_list))
# Unique Sports
print("📌 Unique Sports:")
sports_list = sorted(df['sport'].dropna().unique().tolist())
print(sports_list, "\n")
print(len(sports_list ))

In [ ]:
print("📌 Count of Years:")
year_counts = df['year'].value_counts().sort_index().reset_index()
year_counts.columns = ['Year', 'Count']
print(year_counts)

In [ ]:
print("📌 Count of Sports:")
print(df['sport'].value_counts().sort_index(), "\n")

In [ ]:

# Group by Year and City
summary = df.groupby(['year', 'city']).agg(
    Male=('gender', lambda x: (x == 'M').sum()),
    Female=('gender', lambda x: (x == 'F').sum()),
    Sports=('sport', 'nunique'),   # count unique sports
    Gold=('gold', 'sum'),
    Silver=('silver', 'sum'),
    Bronze=('bronze', 'sum')
).reset_index()
print(summary)
summary.to_csv("Olympics_Summary.csv", index=False)


In [ ]:
df['total_medals'] = df['gold'] + df['silver'] + df['bronze']


In [ ]:
medals_by_gender = df.pivot_table(
    values='total_medals',
    index='year',
    columns='gender',
    aggfunc='sum',
    fill_value=0
).reset_index()
print(medals_by_gender)

In [ ]:
sport_summary = df.groupby(['year', 'sport']).agg(
    Athletes=('name', 'nunique'),
    Male=('gender', lambda x: (x == 'M').sum()),
    Female=('gender', lambda x: (x == 'W').sum()),
    Gold=('gold', 'sum'),
    Silver=('silver', 'sum'),
    Bronze=('bronze', 'sum')
).reset_index()
sport_summary.to_csv("sport_summary.csv", index=False)
sport_summary

In [ ]:
top_sports = df.groupby('sport')['total_medals'].sum().sort_values(ascending=False).head(10)
top_sports

In [ ]:
top_athletes = df.groupby('name')['total_medals'].sum().sort_values(ascending=False).head(10)
top_athletes

In [ ]:

df['gender'] = df['gender'].replace({'F': 'W'})
# Create a Total Medals column
df['total_medals'] = df['gold'] + df['silver'] + df['bronze']

# Filter only athletes with at least one medal
medal_winners = df[df['total_medals'] > 0]

# Group by athlete name and sum medals
medal_summary = medal_winners.groupby('name').agg(
    Gold=('gold', 'sum'),
    Silver=('silver', 'sum'),
    Bronze=('bronze', 'sum'),
    Total=('total_medals', 'sum')
).reset_index()

# Sort by Total medals descending
medal_summary = medal_summary.sort_values(by='Total', ascending=False)

# Display top 10
print(medal_summary.head(10))

# Optional: Save to CSV
medal_summary.to_csv("medal_winners.csv", index=False)
